In [37]:
import numpy as np
import pandas as pd
import math

In [14]:
np.unique([[[1, 1], [1, 0]], [[1, 1], [1, 0]]])

array([0, 1])

In [20]:
def gini_index(groups):
    n_instances = float(sum([len(group) for group in groups]))
    unique_labels = np.unique(groups)
    gini = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        score = 0.0
        for label in unique_labels:
            p = [row[-1] for row in group].count(label) / size
            score += p * p
        gini += (1.0 - score) * (size / n_instances) 
    return gini

In [21]:
print(gini_index([[[1, 1], [1, 0]], [[1, 1], [1, 0]]]))

0.5


In [38]:
def calculate_entropy(y):
    y = np.array(y)
    log2 = lambda x: math.log(x) / math.log(2)
    unique_labels = np.unique(y)
    entropy = 0
    
    for label in unique_labels:
        count = len(y[y == label])
        p = count / len(y)
        entropy += -p * log2(1 - p)
    return entropy

In [39]:
print(calculate_entropy([1, 1, 1, 0, 1, 1, 1, 0]))

1.603759374819711


## Create a Split

In [41]:
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

In [45]:
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0]) - 1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

In [49]:
dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]
split = get_split(dataset)
print('Split: [X%d < %.3f]' % ((split['index']+1), split['value']))
print(split['groups'])

Split: [X1 < 6.642]
([[2.771244718, 1.784783929, 0], [1.728571309, 1.169761413, 0], [3.678319846, 2.81281357, 0], [3.961043357, 2.61995032, 0], [2.999208922, 2.209014212, 0]], [[7.497545867, 3.162953546, 1], [9.00220326, 3.339047188, 1], [7.444542326, 0.476683375, 1], [10.12493903, 3.234550982, 1], [6.642287351, 3.319983761, 1]])


In [71]:
def to_terminal(group):
    outcomes = [row[-1] for row in group]
#     print(outcomes)
    return max(set(outcomes), key=outcomes.count)

In [69]:
def split(node, max_depth, min_size, depth):
    if node:
        left, right = node['groups']
        del(node['groups'])
        if not left or not right:
            node['left'] = node['right'] = to_terminal(left + right)
            return
        if depth >= max_depth:
            node['left'] = node['right'] = to_terminal(left), to_terminal(right)
            return
        if len(left) <= min_size:
            node['left'] = to_terminal(left)
        else:
            node['left'] = get_split(left)
            split(node['left'], max_depth, min_size, depth + 1)

        if len(right) <= min_size:
            node['right'] = to_terminal(right)
        else:
            node['right'] = to_terminal(right)
            split(node['right'], max_depth, min_size, depth + 1)

In [60]:
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

In [61]:
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))

In [72]:
dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]
tree = build_tree(dataset, 2, 1)
print_tree(tree)

TypeError: 'int' object is not subscriptable